In [63]:
# Simple example
%run sortableList.py

from ipywidgets import widgets
from IPython.display import display
import ipyvuetify as v


items = [{ "name": 'Jane Adams',   "email": 'jane@adams.com'   },
         { "name": 'Paul Davis',   "email": 'paul@davis.com'   },
         { "name": 'Amanda Brown', "email": 'amanda@brown.com' }
        ]

# Creation of a new item
def itemNew():
    return {"name": "new", "email": "empty"}


# Content of an item
def itemContent(item):
    return [
        v.CardSubtitle(class_="mb-n4", children=[item['name']]),
        v.CardText(    class_="mt-n2", children=[item['email']])
    ]

s = sortableList(items=items, width=400, dark=False, allowNew=True, itemNew=itemNew, newOnTop=True, itemContent=itemContent)
s.draw()

Html(children=[VBox(children=(Output(layout=Layout(max_height='600px')), Output()))], tag='div')

In [65]:
s.items

[{'name': 'new', 'email': 'empty'},
 {'name': 'Jane Adams', 'email': 'jane@adams.com'},
 {'name': 'Paul Davis', 'email': 'paul@davis.com'},
 {'name': 'Amanda Brown', 'email': 'amanda@brown.com'}]

In [2]:
%run switch.py
%run datePicker.py
%run sortableList.py
%run tooltip.py

from ipywidgets import widgets
from IPython.display import display
import ipyvuetify as v

output = widgets.Output()

items = [{ "name": "Paul",    "surname": "Dockery",  "married": False, "date": "" },
         { "name": "July",    "surname": "Winters",  "married": True,  "date": "1997-07-28" },
         { "name": "David",   "surname": "Forest",   "married": True,  "date": "1999-03-03" },
         { "name": "Dorothy", "surname": "Landmann", "married": False, "date": "" }
        ]

dark = False

# Called when an item is moved or deleted
def onchange():
    with output:
        print('Changed!')

    
# Creation of a new item
def itemNew():
    return { "name": "", "surname": "", "married": False, "date": "" }


# Remove all items
def itemRemoveAll(widget, event, data):
    s.items = []

reset = v.Btn(icon=True, dark=dark, children=[v.Icon(children=['mdi-playlist-remove'])])
reset.on_event('click', itemRemoveAll)


# Content of an item
def itemContent(item):
    
    def onname(widget, event, data):
        item["name"] = int(data)
    
    def onsurname(widget, event, data):
        item["surname"] = data
    
    def onmarried(flag):
        item["married"] = flag
        dp.disabled = not flag
        if not flag:
            item["date"] = ''
            dp.date = None
            
    def ondate():
        item["date"] = dp.date

    tfname = v.TextField(label='Name:', value=item['name'], color='amber', dense=True, style_="max-width: 70px", class_="pa-0 ma-0 mt-2")
    tfname.on_event('input', onname)
    
    tfsurname = v.TextField(label='Surname:', value=item['surname'], color='amber', dense=True, style_="max-width: 100px", class_="pa-0 ma-0 mt-2")
    tfsurname.on_event('input', onsurname)
    
    sw = switch(item['married'], "Married", onchange=onmarried)
    
    dp = datePicker(date=item['date'], dark=dark, width=88, onchange=ondate, offset_x=True, offset_y=False)
    dp.disabled = not item['married']
    
    sp = v.Html(tag='div', class_="pa-0 ma-0 mr-3", children=[''])
    
    return [ v.Row(class_="pa-0 ma-0 ml-2", no_gutters=True, children=[tfname, sp, tfsurname, sp, sw.draw(), sp, dp.draw()]) ]


s = sortableList(items=items, width=520, outlined=False, dark=dark,
                 allowNew=True, itemNew=itemNew, itemContent=itemContent,
                 bottomContent=[tooltip("Remove all persons",reset)],
                 onchange=onchange, buttonstooltip=True)
display(s.draw())
display(output)

Html(children=[VBox(children=(Output(layout=Layout(max_height='600px')), Output()))], tag='div')

Output()

In [ ]:
s.items

In [111]:
# Simple example with one of the item always active
%run sortableList.py
%run tooltip.py

from ipywidgets import widgets
from IPython.display import display
import ipyvuetify as v


items = [{ "name": 'Jane Adams',   "email": 'jane@adams.com'   },
         { "name": 'Paul Davis',   "email": 'paul@davis.com'   },
         { "name": 'Amanda Brown', "email": 'amanda@brown.com' }
        ]

# Creation of a new item
def itemNew():
    return {"name": "new", "email": "empty"}


# Content of an item
def itemContent(item):
    return [
        v.CardSubtitle(class_="mb-n4", children=[item['name']]),
        v.CardText(    class_="mt-n2", children=[item['email']])
    ]

# An item is activated
def onactivated(index):
    #print("Activated %d"%index)
    s.cards[index].children[1].style_ = "font-weight: 800;"  # N.B.: Each card has a CardTitle children followed by the widgets returned by the itemContent function!!!
    s.cards[index].children[2].style_ = "font-weight: 800;"

# An item is deactivated
def ondeactivated(index):
    #print("Deactivated %d"%index)
    s.cards[index].children[1].style_ = "font-weight: 400;"
    s.cards[index].children[2].style_ = "font-weight: 400;"


# Remove all items
def itemRemoveAll(widget, event, data):
    s.items = []

reset = v.Btn(icon=True, dark=dark, children=[v.Icon(children=['mdi-playlist-remove'])])
reset.on_event('click', itemRemoveAll)
    
# Removing an item
removeditem_wasactive = False   # True if the removed item was the active one
def onremoving(index):
    global removeditem_wasactive
    removeditem_wasactive = False
    if index == s.active:
        removeditem_wasactive = True
        
# Remove an item
def onremoved(index):
    if removeditem_wasactive:
        s.active = 0   # keep an item active!

# New item added
def onadded(index):
    if s.active < 0:
        s.active = 0   # keep an item active!
        
s = sortableList(items=items, width=400, maxheightlist=900, dark=False, allowNew=True, itemNew=itemNew, newOnTop=True, itemContent=itemContent,
                 bottomContent=[tooltip("Remove all layers",reset)], buttonstooltip=True, onremoving=onremoving, onremoved=onremoved, onadded=onadded,
                 tooltipadd='Add new layer', tooltipdown='Move layer down', tooltipup='Move layer up', tooltipremove='Remove layer',
                 activatable=True, ondeactivated=ondeactivated, onactivated=onactivated)
s.active = 0
s.draw()

Html(children=[VBox(children=(Output(layout=Layout(max_height='900px')), Output()))], tag='div')

In [106]:
s.active

1

In [75]:
s.active = 0